In [5]:
import arcpy
import pandas as pd
from arcpy import env

# Set the workspace
env.workspace = "masters\VSFB_site_selection\VSFB_Sites\Default.gdb"

# Specify the feature class
fc = 'monarch_sites_VSFB'

# Get the field names
field_names = [f.name for f in arcpy.ListFields(fc)]

# Create a pandas dataframe from the feature class
data = [i for i in arcpy.da.SearchCursor(fc, field_names)]
df = pd.DataFrame(data, columns=field_names)

# Normalize the columns
df['average_10yr_normalized'] = (df['average_10yr'] - df['average_10yr'].min()) / (df['average_10yr'].max() - df['average_10yr'].min())
df['stdDev_10yr_normalized'] = (df['stdDev_10yr'] - df['stdDev_10yr'].min()) / (df['stdDev_10yr'].max() - df['stdDev_10yr'].min())
df['count_10yr_normalized'] = (df['count_10yr'] - df['count_10yr'].min()) / (df['count_10yr'].max() - df['count_10yr'].min())

# Calculate the ranking score
df['ranking_score'] = (df['average_10yr_normalized'] + 
                       (1 - df['stdDev_10yr_normalized']) + 
                       df['count_10yr_normalized'])

# Sort by the ranking score in descending order
df = df.sort_values('ranking_score', ascending=False)

# Print the ranked sites
print(df[['SiteID', 'SiteName', 'ranking_score']])



ModuleNotFoundError: No module named 'arcpy'

In [3]:
conda install arcpy=3.1 -c esri

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.

PackagesNotFoundError: The following packages are not available from current channels:

  - arcpy=3.1

Current channels:

  - https://conda.anaconda.org/esri/osx-64
  - https://conda.anaconda.org/esri/noarch
  - https://conda.anaconda.org/conda-forge/osx-64
  - https://conda.anaconda.org/conda-forge/noarch
  - https://repo.anaconda.com/pkgs/main/osx-64
  - https://repo.anaconda.com/pkgs/main/noarch
  - https://repo.anaconda.com/pkgs/r/osx-64
  - https://repo.anaconda.com/pkgs/r/noarch

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.



Note: you may need to restart the kernel to use updated packages.


In [3]:
print(df.columns)



Index(['OBJECTID', 'Shape', 'SiteID', 'SiteName', 'COUNTYNAME', 'Latitude',
       'Longitude', 'IsWMTCSite', 'GlobalID', 'SiteIDText', 'RegionID',
       'F1997', 'F1998', 'F1999', 'F2000', 'F2001', 'F2002', 'F2003', 'F2004',
       'F2005', 'F2006', 'F2007', 'F2008', 'F2009', 'F2010', 'F2011', 'F2012',
       'F2013', 'F2014', 'F2015', 'F2016', 'NY_F2016', 'F2017', 'NY_F2017',
       'F2018', 'NY_F2018', 'F2019', 'NY_F2019', 'F2020', 'NY_F2020', 'F2021',
       'NY_F2021', 'F2022', 'NY_F2022', 'Shape_Length', 'Shape_Area',
       'max_value', 'max_year', 'count_variance', 'mad', 'Score',
       'median_10yr', 'mad_10yr', 'score_10yr', 'average_10yr', 'count_10yr',
       'stdDev_10yr', 'average_10yr_normalized', 'stdDev_10yr_normalized',
       'count_10yr_normalized', 'ranking_score'],
      dtype='object')


In [5]:
# Add a new field to store the ranking score
arcpy.AddField_management(fc, 'RankingScore', 'FLOAT')

# Create a dictionary mapping SiteID to ranking_score
ranking_dict = pd.Series(df.ranking_score.values, index=df.SiteID).to_dict()

# Update the RankingScore field with the ranking scores
with arcpy.da.UpdateCursor(fc, ['SiteID', 'RankingScore']) as cursor:
    for row in cursor:
        # Only update the row if the SiteID is in the ranking_dict
        if row[0] in ranking_dict:
            row[1] = ranking_dict[row[0]]
            cursor.updateRow(row)
